In [27]:
from multiprocessing import context
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings, Document
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.core.prompts import RichPromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

Settings.api_key = GOOGLE_API_KEY
Settings.embed_model = GoogleGenAIEmbedding(model_name="gemini-embedding-001")
Settings.llm = GoogleGenAI(
    model="gemini-2.5-flash",
    temperature=0.0
)

# Define the custom system prompt
custom_prompt = """
Sua tarefa é responder a perguntas sobre as estratégias de trading com base no contexto fornecido.
Para cada resposta, forneça a galeria de fotos (galeryId) relevante no formato JSON.
Não inclua markdown.
Se a galeria não for mencionada no contexto, responda apenas com a informação textual.
Formato de saída:
[{
  "textBlock": "Sua resposta textual aqui.",
  "galeryId": <o ID da galeria relevante ou null>
},{
  "textBlock": "Sua resposta textual aqui.",
  "galeryId": <o ID da galeria relevante ou null>
},{
  "textBlock": "Sua resposta textual aqui.",
  "galeryId": <o ID da galeria relevante ou null>
},]

Context information is below.
---------------------
{{context_str}}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {{query_str}}
Answer: 
"""

data = {
    "name": "Opening Range Breakout",
    "description": "Trade de rompimento de range de abertura",
    "content": [
        [{"text":"1. Depois que fechar fora do range", "galeryId": 1},
        {"text":"2. O range começa quando a maxima da barra anterior for maior que a barra atual. Como na barra 1. Ela marca o inicio do range. A barra 3 forma o segundo ponto do range, o de venda.", "galeryId": 2},
        {"text":"3. Veja que a barra 2 não fechou fora do range portanto não é uma entrada", "galeryId": None},
        {"text":"4. A barra 4 fechou fora, então é uma entrada.", "galeryId": None}]
    ]
}

strategy_text = " ".join(
    f'{item["text"]} {{"galeryId":{item["galeryId"] if item["galeryId"] is not None else "null"}}}'
    for sublist in data["content"] for item in sublist
)

# Create a Document object
strategy_document = Document(
    text=strategy_text,
    metadata={
        "name": data["name"],
        "description": data["description"]
    }
)

# Create a VectorStoreIndex from the documents
index = VectorStoreIndex.from_documents([strategy_document])

# Use the custom system prompt when creating the query engine
qa_prompt_tmpl = RichPromptTemplate(custom_prompt)
query_engine = index.as_query_engine(text_qa_template=qa_prompt_tmpl)
response = query_engine.query("Quais as regra do trade?")
print(response)

[{
  "textBlock": "O trade ocorre depois que a barra fechar fora do range.",
  "galeryId": 1
},{
  "textBlock": "O range começa quando a máxima da barra anterior for maior que a barra atual (ex: barra 1). A barra 3 forma o segundo ponto do range, o de venda.",
  "galeryId": 2
},{
  "textBlock": "Uma barra que não fecha fora do range não é uma entrada (ex: barra 2).",
  "galeryId": null
},{
  "textBlock": "Uma barra que fecha fora do range é uma entrada (ex: barra 4).",
  "galeryId": null
}]


In [26]:
print(response)

[{
  "textBlock": "O range começa quando a máxima da barra anterior for maior que a barra atual. A barra 1 marca o início do range, e a barra 3 forma o segundo ponto do range, o de venda.",
  "galeryId": 2
},{
  "textBlock": "A entrada ocorre depois que a barra fechar fora do range.",
  "galeryId": 1
},{
  "textBlock": "Por exemplo, a barra 2 não fechou fora do range, portanto não é uma entrada.",
  "galeryId": null
},{
  "textBlock": "A barra 4 fechou fora, então é uma entrada.",
  "galeryId": null
}]


In [8]:

from IPython.display import Markdown, display
query_engine = index.as_query_engine()
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))
display_prompt_dict(query_engine.get_prompts())

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>